In [104]:
pip install python-bitcoinrpc

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [105]:
from bitcoinrpc.authproxy import AuthServiceProxy, JSONRPCException

# Replace these with your actual RPC credentials from bitcoin.conf
rpc_user = "dini"
rpc_password = "qKbD4TBjrwlZGlECoakKp95IrpE5d0RTRZD0I4mTLMo"
rpc_host = "127.0.0.1"
rpc_port = 18443

# Create an RPC connection
rpc_client = AuthServiceProxy(f"http://{rpc_user}:{rpc_password}@{rpc_host}:{rpc_port}", timeout=120)

# Function to ensure wallet is loaded or created
def ensure_wallet_loaded(wallet_name):
    try:
        # Check if wallet is already loaded
        loaded_wallets = rpc_client.listwallets()
        if wallet_name in loaded_wallets:
            print(f"✅ Wallet '{wallet_name}' is already loaded.")
            return

        # Try loading the wallet
        print(f"🔄 Attempting to load wallet '{wallet_name}'...")
        rpc_client.loadwallet(wallet_name)
        print(f"✅ Wallet '{wallet_name}' loaded successfully.")

    except JSONRPCException as e:
        if e.error.get("code") == -18:  # Wallet does not exist
            print(f"⚠️ Wallet '{wallet_name}' does not exist. Creating it...")
            rpc_client.createwallet(wallet_name)
            print(f"✅ Wallet '{wallet_name}' created and loaded successfully.")
        else:
            print(f"❌ Unexpected error: {e}")
            raise

# Ensure the wallet is loaded or created
wallet_name = "testwallet"
ensure_wallet_loaded(wallet_name)

# Reconnect to the wallet after ensuring it is loaded
rpc_client = AuthServiceProxy(f"http://{rpc_user}:{rpc_password}@{rpc_host}:{rpc_port}/wallet/{wallet_name}")

# Generate legacy addresses
address_A = rpc_client.getnewaddress("A", "legacy")
address_B = rpc_client.getnewaddress("B", "legacy")
address_C = rpc_client.getnewaddress("C", "legacy")

print("Address A:", address_A)
print("Address B:", address_B)
print("Address C:", address_C)


✅ Wallet 'testwallet' is already loaded.
Address A: ms8e88ziKS2pJ7L5EsMg5F2kKp7tZ58d1W
Address B: mvq6usiYJHo5pwgL9ktyfDLGzJtJ7LMTYm
Address C: miKmdSwXDgeoHMoxa5SaPr4cGyqtxerThD


In [106]:
# Fund address A
rpc_client.sendtoaddress(address_A, 0.01)


'55ecada0936d01baccaea26ff46f72c03543e7f3225e9a26f4a2d9a3e22589f2'

In [107]:
from decimal import Decimal

# Get the unspent outputs for address A
unspent_outputs = rpc_client.listunspent(0, 9999999, [address_A])

# Select the first unspent output
utxo = unspent_outputs[0]

# Create a raw transaction
raw_tx = rpc_client.createrawtransaction(
    [{"txid": utxo["txid"], "vout": utxo["vout"]}],
    {address_B: utxo["amount"] - Decimal("0.0001")}
)

# Decode the raw transaction
decoded_raw_tx = rpc_client.decoderawtransaction(raw_tx)
print(decoded_raw_tx)

# Sign the transaction using the wallet
signed_tx = rpc_client.signrawtransactionwithwallet(raw_tx)

# Decode the signed transaction
decoded_signed_tx = rpc_client.decoderawtransaction(signed_tx['hex'])
print("\nSigned Transaction from A to B:")
print(decoded_signed_tx)
signed_tx_hex = signed_tx['hex']

# Test mempool acceptance
test_result = rpc_client.testmempoolaccept([signed_tx_hex])

# Check if transaction is valid
if test_result[0]['allowed']:
    print("✅ Transaction is valid and will be accepted by the mempool.")
else:
    print(f"❌ Transaction is invalid. Reason: {test_result[0]['reject-reason']}")

# Broadcast the transaction
txid_A_to_B = rpc_client.sendrawtransaction(signed_tx['hex'])
print("Transaction ID from A to B:", txid_A_to_B)


{'txid': '9de70a22326daf502981fb512a4f9ec2ad78fd111dc9f21657dfd49591faa39c', 'hash': '9de70a22326daf502981fb512a4f9ec2ad78fd111dc9f21657dfd49591faa39c', 'version': 2, 'size': 85, 'vsize': 85, 'weight': 340, 'locktime': 0, 'vin': [{'txid': '55ecada0936d01baccaea26ff46f72c03543e7f3225e9a26f4a2d9a3e22589f2', 'vout': 0, 'scriptSig': {'asm': '', 'hex': ''}, 'sequence': 4294967293}], 'vout': [{'value': Decimal('0.00990000'), 'n': 0, 'scriptPubKey': {'asm': 'OP_DUP OP_HASH160 a7f89153ce4c266634808442b766c0b74147e13e OP_EQUALVERIFY OP_CHECKSIG', 'desc': 'addr(mvq6usiYJHo5pwgL9ktyfDLGzJtJ7LMTYm)#pn7zrk86', 'hex': '76a914a7f89153ce4c266634808442b766c0b74147e13e88ac', 'address': 'mvq6usiYJHo5pwgL9ktyfDLGzJtJ7LMTYm', 'type': 'pubkeyhash'}}]}

Signed Transaction from A to B:
{'txid': '5fa68d7c34f0fc22871e675e8ba9072117b6485c2b50eee699cae4624255c6d9', 'hash': '5fa68d7c34f0fc22871e675e8ba9072117b6485c2b50eee699cae4624255c6d9', 'version': 2, 'size': 191, 'vsize': 191, 'weight': 764, 'locktime': 0, 'vi

In [108]:
# Get the unspent outputs for address B
unspent_outputs_B = rpc_client.listunspent(0, 9999999, [address_B])

# Ensure there are UTXOs available
if not unspent_outputs_B:
    raise ValueError("No UTXOs available for address B!")

# Select the first unspent output (UTXO from A → B)
utxo_B = unspent_outputs_B[0]
txid_A_to_B = utxo_B["txid"]

# Retrieve the transaction A → B to extract scriptPubKey
tx_A_to_B = rpc_client.getrawtransaction(txid_A_to_B, True)
scriptPubKey_hex_A = tx_A_to_B["vout"][utxo_B["vout"]]["scriptPubKey"]["hex"]

# Create a raw transaction from B to C
fee = Decimal("0.0001")
raw_tx_B = rpc_client.createrawtransaction(
    [{"txid": utxo_B["txid"], "vout": utxo_B["vout"]}],
    {address_C: utxo_B["amount"] - fee}
)

# Decode the raw transaction
decoded_raw_tx_B = rpc_client.decoderawtransaction(raw_tx_B)
print("Decoded Raw Transaction from B to C:", decoded_raw_tx_B)

# Sign the transaction using the wallet
signed_tx_B = rpc_client.signrawtransactionwithwallet(raw_tx_B)

# Decode the signed transaction
decoded_signed_tx_B = rpc_client.decoderawtransaction(signed_tx_B['hex'])
print("\nSigned Transaction from B to C:")
print(decoded_signed_tx_B)

# Extract scriptSig from B → C (unlocking A's UTXO)
scriptSig_asm_B = decoded_signed_tx_B['vin'][0].get('scriptSig', {}).get('asm', "Missing ScriptSig")

print("\n🔍 Comparing Scripts:")
print(f"scriptSig_asm_B (B unlocking A's UTXO): {scriptSig_asm_B}")
print(f"scriptPubKey_hex_A (Locking script from A → B): {scriptPubKey_hex_A}")



Decoded Raw Transaction from B to C: {'txid': '2952f7010abc57d17ff0359d928dcb3998d6ac71312dbe4d074854ba372d6990', 'hash': '2952f7010abc57d17ff0359d928dcb3998d6ac71312dbe4d074854ba372d6990', 'version': 2, 'size': 85, 'vsize': 85, 'weight': 340, 'locktime': 0, 'vin': [{'txid': '5fa68d7c34f0fc22871e675e8ba9072117b6485c2b50eee699cae4624255c6d9', 'vout': 0, 'scriptSig': {'asm': '', 'hex': ''}, 'sequence': 4294967293}], 'vout': [{'value': Decimal('0.00980000'), 'n': 0, 'scriptPubKey': {'asm': 'OP_DUP OP_HASH160 1ecae6080de6340eab07e7eac61b25e22b272885 OP_EQUALVERIFY OP_CHECKSIG', 'desc': 'addr(miKmdSwXDgeoHMoxa5SaPr4cGyqtxerThD)#3m26aq38', 'hex': '76a9141ecae6080de6340eab07e7eac61b25e22b27288588ac', 'address': 'miKmdSwXDgeoHMoxa5SaPr4cGyqtxerThD', 'type': 'pubkeyhash'}}]}

Signed Transaction from B to C:
{'txid': '6c7a2900581674b371637581a8d274f0369bdde5e12ff90c9ad1c0ad0158b1f2', 'hash': '6c7a2900581674b371637581a8d274f0369bdde5e12ff90c9ad1c0ad0158b1f2', 'version': 2, 'size': 191, 'vsize': 1